In [1]:
import json
with open('rentCollectionData.json') as d:
    raw = d.read()
raw = '[' + raw + ']'
rentCollect = json.loads(raw)

with open('addresses.json') as d:
    raw2 = d.read()
raw2 = '[' + raw2 + ']'
address = json.loads(raw2)

with open('cleanlistings.json') as d:
    raw3 = d.read()
raw3 = '[' + raw3 + ']'
craig = json.loads(raw3)

In [2]:
rentCollect = rentCollect[0]['housing']
address = address[0]['addrs']
craig = craig[0]['listings']

In [3]:
address[0]

['2171', 'Allston Way', 37.8698901, -122.2661499]

In [4]:
# clean rentCollect
rent = []
for i in range(len(rentCollect)):
    temp = {}
    temp['number'] = rentCollect[i]['number']
    temp['street'] = rentCollect[i]['name'] 
    temp['avg0'] = rentCollect[i]['avg0']
    temp['avg1'] = rentCollect[i]['avg1']
    temp['avg2'] = rentCollect[i]['avg2']
    temp['avg3'] = rentCollect[i]['avg3']
    temp['avg4plus'] = rentCollect[i]['avg4plus']
    rent.append(temp)

In [5]:
# clean address
loc = []
for i in range(len(address)):
    temp = {}
    temp['number'] = address[i][0]
    temp['street'] = address[i][1] 
    temp['latitude'] = address[i][2]
    temp['longitude'] = address[i][3]
    loc.append(temp)

In [6]:
import pandas as pd
rent_df = pd.DataFrame(rent)
loc_df = pd.DataFrame(loc)
craig_df = pd.DataFrame(craig)

In [7]:
df = pd.merge(rent_df, loc_df)
craig_df = craig_df.drop(['_id', 'link', 'numImages', 'postingDate'], 1)

In [8]:
df1 = df.drop_duplicates(subset = 'latitude')
df2 = craig_df.drop_duplicates('latitude')

In [9]:
result = pd.concat([df1, df2])
result = result.drop_duplicates('latitude')

In [10]:
result = result.reset_index(drop = True)
result = result.fillna(value = 0)

In [12]:
# address
for i in range(2291):
    if result['address'][i] == 0:
        result['address'][i] = str(result['number'][i]) + ' ' + str(result['street'][i])
        

/Users/yikaluo/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [13]:
result = result.drop(['number', 'street'], 1)
result['price per room'] = pd.Series(0, index=result.index)

In [14]:
# price

for i in range(2291):
    if result['title'][i] != 0:
        if result['bedrooms'][i] == 0:
            result['bedrooms'][i] = 1
        result['price per room'][i] = result['price'][i] / result['bedrooms'][i]
    else:
        total = 0
        num = 0
        if result['avg0'][i] != 0:
            total += result['avg0'][i]
            num += 1
        if result['avg1'][i] != 0:
            total += result['avg1'][i]
            num += 1
        if result['avg2'][i] != 0:
            total += result['avg2'][i]
            num += 1
        if result['avg3'][i] != 0:
            total += result['avg3'][i]
            num += 1
        result['price per room'][i] = total / num

/Users/yikaluo/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/yikaluo/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/yikaluo/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [17]:
result = result.drop(['avg0', 'avg1', 'avg2', 'avg3', 'avg4plus', 'price'], 1)

In [18]:
result.to_csv("clean_rent2.csv")